In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
db_user = ""
db_password = ""
db_host = ""  
db_port = "" 
db_name = ""
connection = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(connection)

In [3]:
query= text("SELECT DATE(datefrom) AS date, toid, SUM(datavalue) AS datavalue FROM movements GROUP BY date, toid ORDER BY date;")
try:
    with engine.connect() as connection:
        chunks = pd.read_sql(query, connection, chunksize=1000) 
        df = pd.concat(chunks, ignore_index=True) 
    print(df)
except Exception as e:
    print(f"Errore durante l'esecuzione della query: {e}")

             date                toid  datavalue
0      2019-08-01  08|033|001|000|000       3649
1      2019-08-01  08|033|002|000|000       7193
2      2019-08-01  08|033|003|000|000       2245
3      2019-08-01  08|033|004|000|000       6263
4      2019-08-01  08|033|005|000|000       3571
...           ...                 ...        ...
28543  2019-09-30  08|099|024|000|000       4078
28544  2019-09-30  08|099|025|000|000      10383
28545  2019-09-30  08|099|026|000|000       2919
28546  2019-09-30  08|099|027|000|000       2315
28547  2019-09-30  08|099|999|000|255      55142

[28548 rows x 3 columns]


In [4]:
from sklearn.preprocessing import LabelEncoder
df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d")
df["weekday"] = df["date"].dt.weekday
df["week"] = df["date"].dt.isocalendar().week - df["date"].dt.isocalendar().week.min()
df['weekend'] = df['weekday'].apply(lambda w: 1  if (w == 5 or w == 6) else 0)
df["month"] = df["date"].dt.month.map({8: 0, 9: 1})
df['festivo'] = ((df['week'].isin([1, 2])) | (df['weekday'].isin([5, 6]))).astype(int)

df

,date,toid,datavalue,weekday,week,weekend,month,festivo
0,2019-08-01,08|033|001|000|000,3649,3,0,0,0,0
1,2019-08-01,08|033|002|000|000,7193,3,0,0,0,0
2,2019-08-01,08|033|003|000|000,2245,3,0,0,0,0
3,2019-08-01,08|033|004|000|000,6263,3,0,0,0,0
4,2019-08-01,08|033|005|000|000,3571,3,0,0,0,0
...,...,...,...,...,...,...,...,...
28543,2019-09-30,08|099|024|000|000,4078,0,9,0,1,0
28544,2019-09-30,08|099|025|000|000,10383,0,9,0,1,0
28545,2019-09-30,08|099|026|000|000,2919,0,9,0,1,0
28546,2019-09-30,08|099|027|000|000,2315,0,9,0,1,0


In [5]:
ace = '08|037|025' # Dozza 

Raggruppiamo le righe che iniziano con 'ace'

In [7]:
# 1. Seleziona solo le righe che iniziano con il prefisso
df_to_aggregate = df[df['toid'].astype(str).str.startswith(ace)].copy()

# 2. Raggruppa per date sommando i datavalue
df_aggregated = (
    df_to_aggregate
    .groupby('date', as_index=False)
    .agg({
        'datavalue': 'sum',
        'weekday': 'first',
        'week': 'first',
        'weekend': 'first',
        'festivo': 'first',
        'month' : 'first'
    })
)

# 3. Aggiungi un toid rappresentativo per il gruppo aggregato
df_aggregated['toid'] = ace + '|000|000'

# 4. Rimuovi dal dataframe originale le righe che fanno parte del gruppo
df_rest = df[~df['toid'].astype(str).str.startswith(ace)].copy()

# 5. Unisci tutto: righe non aggregate + riga aggregata per giorno
df = pd.concat([df_rest, df_aggregated], ignore_index=True).sort_values(['date', 'toid']).reset_index(drop=True)


In [8]:
# 6. Aggiungi le lag per ciascun toid separatamente
for lag in [1, 2, 3, 7]:
    df[f'lag_{lag}'] = (
        df.sort_values(['toid', 'date'])  # Ordina prima
          .groupby('toid')['datavalue']   # Gruppo per toid
          .shift(lag)                     # Calcola lag
    )

# 7. Sostituisci i NaN (primi giorni) con 0
df[['lag_1', 'lag_2', 'lag_3', 'lag_7']] = df[[
    'lag_1', 'lag_2', 'lag_3', 'lag_7'
]].fillna(0)
df

,date,toid,datavalue,weekday,week,weekend,month,festivo,lag_1,lag_2,lag_3,lag_7
0,2019-08-01,08|033|001|000|000,3649,3,0,0,0,0,0.0,0.0,0.0,0.0
1,2019-08-01,08|033|002|000|000,7193,3,0,0,0,0,0.0,0.0,0.0,0.0
2,2019-08-01,08|033|003|000|000,2245,3,0,0,0,0,0.0,0.0,0.0,0.0
3,2019-08-01,08|033|004|000|000,6263,3,0,0,0,0,0.0,0.0,0.0,0.0
4,2019-08-01,08|033|005|000|000,3571,3,0,0,0,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
28543,2019-09-30,08|099|024|000|000,4078,0,9,0,1,0,4863.0,4672.0,3431.0,3181.0
28544,2019-09-30,08|099|025|000|000,10383,0,9,0,1,0,10122.0,10054.0,9097.0,8889.0
28545,2019-09-30,08|099|026|000|000,2919,0,9,0,1,0,4304.0,3761.0,3149.0,2776.0
28546,2019-09-30,08|099|027|000|000,2315,0,9,0,1,0,2293.0,2454.0,2255.0,2187.0


Prova con diversi seed

In [10]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import  GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

seeds = [0, 7, 13, 21, 42, 99, 123, 34, 67, 80]
results = []

for seed in seeds:
    # 1. Filtro ace
    df_ace = df[df['toid'].astype(str).str.startswith(ace)].copy()

    # 2. Split 70/30 sui giorni
    train_days, test_days = train_test_split(df_ace['date'], test_size=0.3, random_state=seed)

    # 3. Crea train e test set
    train_df = df[~df['date'].isin(test_days)].copy()     # Tutti gli ACE nei giorni NON di test
    test_df = df_ace[df_ace['date'].isin(test_days)].copy()  # Solo Dozza nei giorni di test

    # 4. Encoding e conversioni
    train_df['date'] = train_df['date'].astype('int64')
    test_df['date'] = test_df['date'].astype('int64')

    label_encoder = LabelEncoder()
    train_df['toid'] = label_encoder.fit_transform(train_df['toid'])

    try:
        test_df['toid'] = label_encoder.transform(test_df['toid'])  # può fallire se toid sconosciuti
    except:
        continue  # salta il seed se ci sono toid ignoti

    # 5. Selezione delle feature
    features = ['date', 'toid', 'lag_1', 'lag_2', 'lag_3', 'lag_7'] 
    X_train = train_df[features]
    y_train = train_df['datavalue']
    X_test = test_df[features]
    y_test = test_df['datavalue']

    # 6. Scaling
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 7. Log-transform target
    y_train_log = np.log1p(y_train)
    y_test_log = np.log1p(y_test)

    # 8. GridSearch
    model = GradientBoostingRegressor(random_state=seed)
    param_grid = {
        'n_estimators': [100, 300, 500],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
    }

    grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=0)
    grid.fit(X_train_scaled, y_train_log)

    # 9. Predizione
    best_model = grid.best_estimator_
    y_pred = np.expm1(best_model.predict(X_test_scaled))
    y_true = np.expm1(y_test_log)

    # 10. Metriche
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100

    results.append({
        'seed': seed,
        'MAE': mae,
        'MAPE (%)': mape,
        'Best Params': grid.best_params_
    })

# Output finale
df_results = pd.DataFrame(results)
print(df_results)
print("MAPE medio:", df_results['MAPE (%)'].mean())


   seed          MAE   MAPE (%)  \
0     0   767.638364   8.965464   
1     7   674.186005   8.472363   
2    13  1078.682544  11.604022   
3    21   743.738914   9.181839   
4    42  1289.123732  14.523039   
5    99   851.008962  10.449695   
6   123  1300.566104  14.343475   
7    34  1337.653288  14.552810   
8    67  1521.791334  16.300475   
9    80   695.785158   7.644824   

                                         Best Params  
0  {'learning_rate': 0.01, 'max_depth': 3, 'n_est...  
1  {'learning_rate': 0.01, 'max_depth': 3, 'n_est...  
2  {'learning_rate': 0.01, 'max_depth': 3, 'n_est...  
3  {'learning_rate': 0.01, 'max_depth': 3, 'n_est...  
4  {'learning_rate': 0.01, 'max_depth': 3, 'n_est...  
5  {'learning_rate': 0.01, 'max_depth': 3, 'n_est...  
6  {'learning_rate': 0.01, 'max_depth': 3, 'n_est...  
7  {'learning_rate': 0.01, 'max_depth': 3, 'n_est...  
8  {'learning_rate': 0.01, 'max_depth': 3, 'n_est...  
9  {'learning_rate': 0.01, 'max_depth': 3, 'n_est...  
MAPE medi